In [2]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/" "LLMs-from-scratch/main/ch02/01_main-chapter-code/" "the-verdict.txt") 
file_path = "the-verdict.txt" 
urllib.request.urlretrieve(url, file_path)
#含义：调用 urlretrieve() 函数，从指定的 url 下载内容，并保存为 file_path 指定的文件。

('the-verdict.txt', <http.client.HTTPMessage at 0x1e8a147b250>)

In [3]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()

print(len(raw_text))

20479


In [4]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [7]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [9]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [10]:
all_words = sorted(set(preprocessed))
# set 不包含重复项，可以用来去重
vocab_size = len(all_words)
print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}
# enumerate的作用是：在遍历一个可迭代对象（如列表）时，
# 既能取到元素本身，又能同时获得元素的索引（下标）。
for i, item in enumerate(vocab.items()):
    print(item)
    if i >=50 :
      	break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [12]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        # vocab是我们通过去重后的分词的一个字典
        # 字典中的键值对都是token：索引
        # 其中包含了一个数据集，按照单词划分的所有token
        #A 将词汇表作为类属性存储，以方便在 encode 和 decode 方法中访问
        #B 创建一个反向词汇表，将token ID 映射回原始的文本token

        self.str_to_int = vocab
        self.int_to_str = {i:s for s ,i in vocab.items()}
        
    def encode(self,text):
        #C 将输入文本转换为token ID
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # 只有当item.strip()不为空字符串的时候添加进列表
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids 
        # 返回索引
    
    def decode(self,ids):
        #C 将输入文本转换为token ID
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) 
        #E 在指定的标点符号前去掉空格                          #E
        return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab)
text =  """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
ids


[1,
 56,
 2,
 850,
 988,
 602,
 533,
 746,
 5,
 1126,
 596,
 5,
 1,
 67,
 7,
 38,
 851,
 1108,
 754,
 793,
 7]

In [14]:
print(tokenizer.decode(ids)),text

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


(None,
 '"It\'s the last he painted, you know," Mrs. Gisburn said with pardonable pride.')

In [15]:
all_tokens = sorted(list(set(preprocessed)))
# all_tokens.append(['<|endoftext|>','<|unk|>'])
#append 是把一个元素整体加到列表末尾，这里你添加的是一个列表，结果就变成了嵌套列表：
all_tokens = sorted(list(set(preprocessed)))
for tok in ['<|unk|>', '<|endoftext|>']:
    if tok not in all_tokens:
        all_tokens.append(tok)
print(len(vocab.items()))
vocab = {token:index for index ,token in enumerate(all_tokens)}
print(len(vocab.items()))

1130
1132


| 方法          | 功能           | 示例                  | 结果                    |
| ----------- | ------------ | ------------------- | --------------------- |
| `append(x)` | 添加一个元素（不展开）  | `lst.append([4,5])` | `lst = [1,2,3,[4,5]]` |
| `extend(x)` | 添加多个元素（展开加入） | `lst.extend([4,5])` | `lst = [1,2,3,4,5]`   |


In [16]:
for i, item in enumerate(list(vocab.items())[-5:]):  # ✅ 先切片，再 enumerate
    print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


In [17]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_idx = vocab
        self.idx_to_str = {index:token for token,index in vocab.items()}
    
    def encode(self,text):
        preprocessed = re.split(r'(<\|.*?\|>|[,.?_!"()\']|--|\s)', text)


        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        # 如果不是空字符 就添加进列表
        preprocessed = [item if item in self.str_to_idx 
                        else '<|unk|>' for item in preprocessed]
        # 用 <|unk|> 替换所有“词表外”的 token，
        # 证后面能顺利用 self.str_to_idx[s] 做查找。
        ids = [self.str_to_idx[s] for s in preprocessed]

        return ids
    
    def decode(self,ids):
        text = " ".join([self.idx_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)                    #B
        return text

In [18]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = ' <|endoftext|> '.join((text1, text2))

text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'

In [19]:
tokenizer = SimpleTokenizerV2(vocab)
index = tokenizer.encode(text)
index

[1130, 5, 355, 1126, 628, 975, 10, 1131, 55, 988, 956, 984, 722, 988, 1130, 7]

In [20]:
tokenizer.decode(index)

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [21]:
print('<|endoftext|>' in vocab)     # True 吗？
print(vocab['<|endoftext|>'])       # 有合法 ID 吗？


True
1131


In [22]:
print(preprocessed[-2:])


['.', '"']


## 2.5 字节对编码（Byte pair encoding）

In [23]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In [24]:
help(tiktoken),dir(tiktoken)

Help on package tiktoken:

NAME
    tiktoken - # This is the public API of tiktoken

PACKAGE CONTENTS
    _educational
    _tiktoken
    core
    load
    model
    registry

VERSION
    0.9.0

FILE
    d:\python\lib\site-packages\tiktoken\__init__.py




(None,
 ['Encoding',
  '__builtins__',
  '__cached__',
  '__doc__',
  '__file__',
  '__loader__',
  '__name__',
  '__package__',
  '__path__',
  '__spec__',
  '__version__',
  '_tiktoken',
  'core',
  'encoding_for_model',
  'encoding_name_for_model',
  'get_encoding',
  'list_encoding_names',
  'model',
  'registry'])

In [25]:
tokenizer = tiktoken.encoding_for_model("gpt-4")

In [26]:
help(tokenizer)

Help on Encoding in module tiktoken.core object:

class Encoding(builtins.object)
 |  Encoding(name: 'str', *, pat_str: 'str', mergeable_ranks: 'dict[bytes, int]', special_tokens: 'dict[str, int]', explicit_n_vocab: 'int | None' = None)
 |  
 |  Methods defined here:
 |  
 |  __getstate__(self) -> 'object'
 |      Helper for pickle.
 |  
 |  __init__(self, name: 'str', *, pat_str: 'str', mergeable_ranks: 'dict[bytes, int]', special_tokens: 'dict[str, int]', explicit_n_vocab: 'int | None' = None)
 |      Creates an Encoding object.
 |      
 |      See openai_public.py for examples of how to construct an Encoding object.
 |      
 |      Args:
 |          name: The name of the encoding. It should be clear from the name of the encoding
 |              what behaviour to expect, in particular, encodings with different special tokens
 |              should have different names.
 |          pat_str: A regex pattern string that is used to split the input text.
 |          mergeable_ranks: A d

In [27]:
vocab_size = tokenizer.n_vocab
print(vocab_size)

100277


In [28]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."

intergers = tokenizer.encode(text,allowed_special = {"<|endoftext|>"})
# 特殊 token 是一个特殊 ID，不能通过普通 BPE 组合得到。
# 例如，在 cl100k_base 或 gpt2 编码中：
# <|endoftext|> 的 ID 是 100257
# 它必须在 vocab 的 特殊 token map 中才能被识别
# 如果没有显式允许，它会被“拆碎”成若干普通 token（普通词+符号）
print(intergers)

[9906, 11, 656, 499, 1093, 15600, 30, 220, 100257, 763, 279, 7160, 32735, 7317, 2492, 315, 1063, 16476, 17826, 13]


In [29]:
intergers = tokenizer.encode(text,disallowed_special=())
#你禁用了所有特殊 token 的限制 → 全部特殊 token 都被当作普通文本处理
#< | end of text | >   ← BPE 分成很多小 token
# ... [30, 83739, 8862, 728, 428, 91, 29, ...]
print(intergers)

[9906, 11, 656, 499, 1093, 15600, 30, 83739, 8862, 728, 428, 91, 29, 763, 279, 7160, 32735, 7317, 2492, 315, 1063, 16476, 17826, 13]


In [30]:
strings = tokenizer.decode(intergers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.'

In [31]:
word = "Akwirw ier"
idx = tokenizer.encode(word)
idx

[32, 29700, 404, 86, 602, 261]

In [32]:
string = tokenizer.decode(idx)
string

'Akwirw ier'

##  2.6 使用滑动窗口进行数据采样

In [33]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

4943


In [34]:
enc_example = enc_text[50:]

In [35]:
context_size = 4
x = enc_example[0:context_size]
y = enc_example[1:context_size+1]
x,y

([323, 9749, 5678, 304], [9749, 5678, 304, 264])

In [36]:
for i in range(1,context_size+1):
    context = enc_example[:i]
    desired = enc_example[i]
    print(context,"---->",desired)


[323] ----> 9749
[323, 9749] ----> 5678
[323, 9749, 5678] ----> 304
[323, 9749, 5678, 304] ----> 264


In [37]:
for i in range(1,context_size+1):
    context = enc_example[:i]
    desired = enc_example[i]
    print(tokenizer.decode(context),"---->",tokenizer.decode([desired]))
    # 输入需要是序列


 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [38]:
import torch 
from torch.utils.data import Dataset,DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        super().__init__()

        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)
        #A 将整个文本进行分词

        for i in range(0,len(token_ids)-max_length,stride):
            #B 使用滑动窗口将书籍分块为最大长度的重叠序列。
            input_chunk = token_ids[i:i+max_length]
            # 这样能够确保最大的长度不超过tokenids的长度
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    #C 返回数据集的总行数

    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
        #D 从数据集中返回指定行

In [39]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
    tokenizer = tiktoken.encoding_for_model("gpt-4")
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        #D 用于预处理的CPU进程数量
        drop_last = drop_last,
        #C drop_last=True会在最后一批次小于指定的batch_size时丢弃该批次，以防止训练期间的损失峰值
    )
    return dataloader

In [40]:
with open("the-verdict.txt","r",encoding='utf-8') as f:
    raw_text = f.read()

data_loader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(data_loader)
next(data_iter)

[tensor([[  40,  473, 1846, 2744]]), tensor([[ 473, 1846, 2744, 3463]])]

In [41]:
next(data_iter)

[tensor([[ 473, 1846, 2744, 3463]]), tensor([[1846, 2744, 3463, 7762]])]

In [42]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[ 4856, 15262,   279,  4623],
        [  596, 10788,   574,   779],
        [ 1306,  1461,    11, 17948],
        [   13,   662,   662,  6905],
        [ 8835,    11,   439,   568],
        [ 4265,  4856,  1093,   311],
        [  832,  1317,   291,   311],
        [  313,  1494,    11,  8009]])

Targets:
 tensor([[15262,   279,  4623,   313],
        [10788,   574,   779,  4173],
        [ 1461,    11, 17948,   555],
        [  662,   662,  6905, 46639],
        [   11,   439,   568, 16392],
        [ 4856,  1093,   311,  3371],
        [ 1317,   291,   311, 16106],
        [ 1494,    11,  8009,  4610]])


## 2.7 构建词嵌入层

In [43]:
input_ids = torch.tensor([2, 3, 5, 1])

In [44]:
vocab_size = 6
output_dim = 3

In [45]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [46]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [47]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## 2.8 位置编码

In [48]:

vocab_size = 100277
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [49]:
max_length = 4
dataloader = create_dataloader_v1(
  	raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)
#size = batch_size*max_length 8*4

Token IDs:
 tensor([[   40,   473,  1846,  2744],
        [ 3463,  7762,   480,   285],
        [22464,  4856,   264, 12136],
        [35201,   313,  4636,   264],
        [ 1695, 12637,  3403,   313],
        [  708,   433,   574,   912],
        [ 2294, 13051,   311,   757],
        [  311,  6865,   430,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [54]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
print(pos_embedding_layer.weight)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
print(pos_embeddings)
#.weight = 所有位置的embedding参数
#pos_embedding_layer(索引) = 按需查表，输出“实际要用的那些位置向量”
# 对于 GPT 模型所使用的绝对嵌入方法，我们只需创建另一个嵌入层，
# 其维度与 token_embedding_layer 的维度相同：

Parameter containing:
tensor([[-1.9533, -1.4118, -0.1707,  ..., -0.0447, -0.2458, -0.2742],
        [ 0.7352, -0.2087,  0.0884,  ...,  0.4836, -0.4149,  1.1965],
        [ 0.6648, -0.6451, -1.1558,  ...,  0.1560,  1.0606, -1.2221],
        [ 0.5973, -0.2016, -0.2678,  ..., -0.1727, -0.6388, -0.2301]],
       requires_grad=True)
torch.Size([4, 256])
tensor([[-1.9533, -1.4118, -0.1707,  ..., -0.0447, -0.2458, -0.2742],
        [ 0.7352, -0.2087,  0.0884,  ...,  0.4836, -0.4149,  1.1965],
        [ 0.6648, -0.6451, -1.1558,  ...,  0.1560,  1.0606, -1.2221],
        [ 0.5973, -0.2016, -0.2678,  ..., -0.1727, -0.6388, -0.2301]],
       grad_fn=<EmbeddingBackward0>)


In [52]:
input_embeddings = pos_embeddings+token_embeddings
input_embeddings.shape

torch.Size([8, 4, 256])